# 深度学习框架

## 包内容

- torch：类似于 Numpy 的通用数组库
- torch.autograd：用于构建计算图形并自动获取梯度
- torch.nn：共享层和损失函数的神经网络库
- torch.optim：具有通用优化算法（SGD、Adam等）的优化包

In [1]:
# 创建Tensor
import torch

# 根据列表数据生成
torch.Tensor([1, 2, 3, 4, 5, 6])

# 根据维度生成
torch.Tensor(2,3)

# 根据已有的张量生成
t = torch.Tensor([[1,2,3],[4,5,6]])

print(t.size())

print(t.shape)

torch.Tensor(t.size())


torch.Size([2, 3])
torch.Size([2, 3])


tensor([[0., 0., 0.],
        [0., 0., 0.]])

In [2]:
# 创建单位矩阵  
print(torch.eye(2,2))

# 创建全0矩阵
print(torch.zeros(2,3))

# 创建全1矩阵
print(torch.ones(2,3))

# 创建随机矩阵
print(torch.randn(2,3))

# 创建等差数列
print(torch.linspace(1,10,4))

# 创建与已有张量形状相同的全0矩阵
print(torch.zeros_like(torch.randn(2,3)))


tensor([[1., 0.],
        [0., 1.]])
tensor([[0., 0., 0.],
        [0., 0., 0.]])
tensor([[1., 1., 1.],
        [1., 1., 1.]])
tensor([[ 0.4929, -0.4592,  1.0768],
        [-0.7646, -1.3746,  1.0486]])
tensor([ 1.,  4.,  7., 10.])
tensor([[0., 0., 0.],
        [0., 0., 0.]])


In [3]:
# 修改张量形状
print(torch.reshape(torch.randn(2,3), (3,2)))

# 转置
print(torch.t(torch.randn(2,3)))



tensor([[-1.5588,  1.8407],
        [ 0.0265, -1.1522],
        [-0.2922,  1.1332]])
tensor([[-2.0460,  0.0788],
        [ 0.4230, -0.2723],
        [-1.9369, -1.1090]])


In [4]:
# 矩阵操作
A = torch.randn(2,3)
B = torch.randn(2,3)

print("矩阵加法")
print(A+B)

print("矩阵乘法")
print(A.mm(B.T))

# 广播
A = torch.randn(2,3)
B = torch.randn(2,1)

print("广播")
print(A+B)

# bmm
# 对批量的3维张量进行矩阵乘法
A = torch.randint(10,(2,2,3))
B = torch.randint(6,(2,3,4))

print("bmm")
print(torch.bmm(A,B))




矩阵加法
tensor([[ 0.8654,  3.0735, -0.2771],
        [ 0.4369, -0.0611, -1.3940]])
矩阵乘法
tensor([[ 2.1183, -0.6892],
        [ 1.2280, -0.5641]])
广播
tensor([[-2.5558, -0.0144, -0.9916],
        [ 0.1632,  0.9256, -0.1912]])
bmm
tensor([[[39, 49, 33, 26],
         [42, 42, 33, 36]],

        [[74, 52, 44, 16],
         [62, 36, 46, 14]]])


# autograd

## 符号微分

将原表达式转换为求导表达式

优：精确数值结果

缺：表达式膨胀

## 数值微分

使用有限差分来近似：

$$
\frac{\partial f(x)}{\partial x_i} \approx \frac{f(x + he_i) - f(x)}{h}, \; where h > 0
$$

优：容易实现

缺：计算结果不精确，计算复杂度高，对h要求高

## 自动微分

所有数值计算由有限的基本运算组成，基本运算的导数表达式是已知的，通过链式法则将数值计算部分组合成整体


### OO模式

1. 利用语言多态性，重载基本运算操作符
2. 将表达式操作类型和输入输出信息，记录到 Tape 中
3. 对 Tape 遍历，并对其中记录的基本运算操作进行微分
4. 把结果通过链式法则进行组合

优：
- 实现简单
- 语言具备多态性
- 易用性高，贴合原生语言

缺：
- 显式的构造 Tape 数据结构和对 Tape 进行读写
- 额外数据结构和操作的引入，不利于高阶微分
- if, while通常难以通过操作符重载

参考： https://zhuanlan.zhihu.com/p/69294347

In [5]:
# 前向传播：
#     x = 2
#     │
#     ▼  (平方)
#     y = x² = 4
#     │
#     ▼  (加3)
#     z = y + 3 = 7
#     │
#     ▼  (乘2)
#     loss = 2z = 14
import torch

# 创建输入张量（叶子节点）
x = torch.tensor(2.0, requires_grad=True)

# 前向计算
y = x ** 2        # y = 4
z = y + 3         # z = 7
loss = 2 * z      # loss = 14

print(f"前向结果: x={x.item()}, y={y.item()}, z={z.item()}, loss={loss.item()}")

前向结果: x=2.0, y=4.0, z=7.0, loss=14.0


In [6]:
# 反向传播
# 目标：计算 d(loss)/d(x)

# 根据链式法则：
# d(loss)/d(x) = d(loss)/d(z) × d(z)/d(y) × d(y)/d(x)

# 逐步计算：
# ┌─────────────────────────────────────────────┐
# │ ① d(loss)/d(z) = d(2z)/d(z) = 2            │
# │ ② d(z)/d(y) = d(y+3)/d(y) = 1              │
# │ ③ d(y)/d(x) = d(x²)/d(x) = 2x = 2×2 = 4    │
# └─────────────────────────────────────────────┘

# 最终梯度：
# d(loss)/d(x) = 2 × 1 × 4 = 8


# 反向传播（从 loss 向 x 传播）：

#     loss = 14
#       │
#       │ d(loss)/d(loss) = 1 (初始梯度)
#       ▼
#     loss = 2z
#       │
#       │ d(loss)/d(z) = 2
#       ▼
#     z = y + 3
#       │
#       │ d(loss)/d(y) = 2 × 1 = 2
#       ▼
#     y = x²
#       │
#       │ d(loss)/d(x) = 2 × 2x = 2 × 4 = 8
#       ▼
#     x = 2 (叶子节点，梯度存储在这里)
import torch

# 创建计算图
x = torch.tensor(2.0, requires_grad=True)
y = x ** 2
z = y + 3
loss = 2 * z

# 反向传播
loss.backward()

# 查看梯度
print(f"\n反向传播结果:")
print(f"x.grad = {x.grad.item()}")  # 应该等于 8

# 验证计算图结构
print(f"\n计算图结构:")
print(f"loss.grad_fn: {loss.grad_fn}")
print(f"z.grad_fn: {z.grad_fn}")
print(f"y.grad_fn: {y.grad_fn}")
print(f"x.grad_fn: {x.grad_fn}")  # None (叶子节点)


反向传播结果:
x.grad = 8.0

计算图结构:
loss.grad_fn: <MulBackward0 object at 0x104c8f070>
z.grad_fn: <AddBackward0 object at 0x104cc3fa0>
y.grad_fn: <PowBackward0 object at 0x104c8f070>
x.grad_fn: None


# 神经网络组件

## 基本模块

- 层：神经网络基本结构，将输入张量转换为输出张量
- 模型：由层构成的网络
- 损失函数：参数学习的目标函数，通过最小化损失函数来学习各种参数
- 优化器：在使损失函数最小值时，涉及优化器

## 训练模型

1. 加载和预处理数据集（torch.utils、torchvision等）。
2. 定义损失函数（回归nn.MSELoss()，分类nn.BCELoss()等）。
3. 定义优化方法（torch.Optimizer）
4. 循环训练模型

    ```
    # 设置为训练模式
    model.train()

    # 梯度清零
    optimizer.zero_grad()

    # 求损失值
    y_prev = model(x)
    loss = loss_fn(f_prev,t_true)

    # 自动求导，反向传播
    loss.backward()

    # 更新参数
    optimizer.step()

    # 循环验证模型
    model.eval()

    # 不跟踪梯度计算损失
    with.torch.no_grad():

    ```
